In [1]:
import pandas as pd
import numpy as np

In [5]:
df=pd.read_csv('terror_gtd.csv',encoding='latin')
df.head()

eventid  iyear  imonth  iday         country_txt  \
0  1.970000e+11   1970       0     0  Dominican Republic   
1  1.970000e+11   1970       0     0              Mexico   
2  1.970000e+11   1970       1     0         Philippines   
3  1.970000e+11   1970       1     0              Greece   
4  1.970000e+11   1970       1     0               Japan   

                    region_txt provstate           city location summary  \
0  Central America & Caribbean       NaN  Santo Domingo      NaN     NaN   
1                North America       NaN    Mexico city      NaN     NaN   
2               Southeast Asia    Tarlac        Unknown      NaN     NaN   
3               Western Europe    Attica         Athens      NaN     NaN   
4                    East Asia       NaN        Fukouka      NaN     NaN   

   success  suicide  nkill  nwound  
0        1        0    1.0     0.0  
1        1        0    0.0     0.0  
2        1        0    1.0     0.0  
3        1        0    NaN     NaN  
4        1        0    NaN     NaN

# Which country had highest attacks since 1970? How many attacks happened in India?

In [6]:
df['country_txt'].value_counts().head(1)

Iraq    18770
Name: country_txt, dtype: int64

# How many attacks happened in India and upto 7 people were killed?

In [119]:
df1=df[(df['country_txt']=='India') & (df['nkill']<=7)]
print('Number of attacks happened in India and upto 7 people were killed:',df1.shape[0])

Number of attacks happened in India and upto 7 people were killed: 9184


# Extract any 4 cities and summary for attacks above

In [129]:
lst=df1['city'].unique()[:4]
df1[df1['city'].isin(lst)].groupby('city').agg({'nkill':'sum','nwound':'sum'})

nkill  nwound
city                     
Bombay       30.0    65.0
Imphal       88.0   352.0
New Delhi   155.0  1271.0
Samastipur    4.0     0.0

In [130]:
df1[df1['city'].isin(lst)][['city','summary']]

city                                            summary
1185     New Delhi                                                NaN
2688    Samastipur  1/2/1975: The Indian Railway Minister, Lalit N...
3780     New Delhi                                                NaN
5251        Bombay                                                NaN
7266        Imphal                                                NaN
...            ...                                                ...
154916      Imphal  11/11/2015: An explosive device detonated in M...
155881      Imphal  12/06/2015: Two explosive devices detonated at...
156248      Imphal  12/17/2015: An explosive device detonated in D...
156433      Imphal  12/22/2015: Assailants shot and killed a baker...
156459      Imphal  12/23/2015: An explosive device planted under ...

[503 rows x 2 columns]

# In a single terror incident in India, find out top 3 cities by number killed

In [28]:
df[(df['country_txt']=='India')][['city','nkill']].sort_values('nkill',ascending=False).head(3)

city  nkill
81000   Mumbai  188.0
96598  Jhargam  115.0
54339   Bombay  115.0

# In a single terror incident in India, find out top 5 cities by number killed and wounded

In [35]:
df[(df['country_txt']=='India')][['city','nkill']].sort_values(['nkill'],ascending=False).head(5)

city  nkill
81000              Mumbai  188.0
96598             Jhargam  115.0
54339              Bombay  115.0
95860  Dantewada district   82.0
56837            Banabari   70.0

In [36]:
df[(df['country_txt']=='India')][['city','nwound']].sort_values(['nwound'],ascending=False).head(5)

city  nwound
81000     Mumbai   817.0
89128     Dispur   235.0
30378    Chennai   200.0
79045  New Delhi   155.0
75769     Mumbai   150.0

# Label all the incidents where the number killed was more than 10 as severe. How many incidents were Severe?

In [56]:
def incident(kills):
    if(kills>10):
        return 'Severe'
    else:
        return 'Not Severe'

In [60]:
lst=list(map(incident,df['nkill']))

In [64]:
set(lst)

{'Not Severe', 'Severe'}

In [71]:
df['Severity']=df.apply(lambda row:incident(row.nkill),axis=1)

In [72]:
df['Severity'].value_counts()

Not Severe    149998
Severe          6774
Name: Severity, dtype: int64

# How many attacks were successful that were suicide attacks? Also, how many suicide attacks were not successful?

In [90]:
print('Number of Successful suicide attacks:',df[(df['suicide']==1) & (df['success']==1)].shape[0])
print('Number of UnSuccessful suicide attacks:',df[(df['suicide']==1) & (df['success']==0)].shape[0])

Number of Successful suicide attacks: 4260
Number of UnSuccessful suicide attacks: 511


# Write a function to label an incident that was both successful and suicidal

In [116]:
def suicidal_success(df):
    return (df['suicide']==1) & (df['success']==1)

In [117]:
df['Successful_Sucidal']=df.apply(lambda row:suicidal_success(row),axis=1)

In [118]:
df['Successful_Sucidal'].value_counts()

False    152512
True       4260
Name: Successful_Sucidal, dtype: int64

# Create a new category representing if the incident occurred in Afghanistan, Pakistan or India as one level of the category and all the other countries as another level. Label all other countries as ROW and new column which contains the new category as "Local' in the data.

In [108]:
def category(x):
    if(x['country_txt'] in ['Afghanistan', 'Pakistan' , 'India']):
        #print(x['country_txt'])
        return 'Af-Pak-India'
    else:
        return 'ROW'

In [109]:
df['Local']=df.apply(lambda row:category(row),axis=1)

# How many incidents happened in Af-Pak-India vs ROW?

In [110]:
df['Local'].value_counts()

ROW             124374
Af-Pak-India     32398
Name: Local, dtype: int64

# List the number of suicides attacks and average kills by Af-Pak-India vs ROW. Rename columns in the output as Average Kills for average kills and Number Incidents for count of suicide attacks.

In [115]:
df.groupby(['Local']).agg({'nkill':np.mean,'suicide' : 'count'}).rename(columns={'nkill':'Average_Kills','suicide':'Number_Incidents'})

Average_Kills  Number_Incidents
Local                                        
Af-Pak-India       2.105574             32398
ROW                2.428727            124374